In [1]:
import os
import subprocess
from types import SimpleNamespace
import tarfile

In this tutorial we will go through the steps to prepare data for input to GHIST. We will use a subset of a dataset from 10x Genomics (Breast Cancer Sample 2 https://www.10xgenomics.com/products/xenium-in-situ/preview-dataset-human-breast). We also provide all the processed data to run GHIST for this dataset in https://drive.google.com/drive/folders/1ebk_2l9dUzT35xgRHMtt3SMSi2rWpVY0?usp=sharing

## Setup - Download data

#### 1. Download the Xenium data subset

Note we provide the H&E image that is aligned to the spatial transcriptomics data for this dataset. This tutorial does not provide the code to align H&E images with Xenium data. Affine transform parameters are available for some datasets. Alternatively, please check out https://www.10xgenomics.com/analysis-guides/he-to-xenium-dapi-image-registration-with-fiji. Manual registration is between H&E and DAPI images is another option.

In [2]:
!gdown https://drive.google.com/uc?id=1HPP_gHdZ8TjDMpMiZOOjmGbBtgkp0RCK

Downloading...
From (original): https://drive.google.com/uc?id=1HPP_gHdZ8TjDMpMiZOOjmGbBtgkp0RCK
From (redirected): https://drive.google.com/uc?id=1HPP_gHdZ8TjDMpMiZOOjmGbBtgkp0RCK&confirm=t&uuid=501d3248-1b3c-4ea9-b5b8-9c32b5b78554
To: /dskh/nobackup/helenf/project_GHIST/_GITHUB_restructure_branch/tutorials/data_processing_demo.zip
100%|██████████████████████████████████████| 69.6M/69.6M [00:02<00:00, 29.5MB/s]


#### 2. Extract the data bundle

The following files from the Xenium bundle are required :
- nucleus_boundaries.csv.gz
- cell_feature_matrix (sometimes it is compressed (e.g., cell_feature_matrix.tar.gz), please ensure the folder is extracted)
- H&E image

In [3]:
outs_dir = "data_processing_demo"

In [4]:
!unzip {outs_dir + ".zip"} -d {outs_dir}

Archive:  data_processing_demo.zip
  inflating: data_processing_demo/nucleus_boundaries.csv.gz  
  inflating: data_processing_demo/cell_feature_matrix.tar.gz  
  inflating: data_processing_demo/he_image.tif  


In [5]:
tar_path = os.path.join(outs_dir, "cell_feature_matrix.tar.gz")
extracted_folder = os.path.join(outs_dir, "cell_feature_matrix")

if os.path.exists(extracted_folder):
    print(f"cell_feature_matrix folder already exists")
elif os.path.exists(tar_path):
    with tarfile.open(tar_path, "r:gz") as tar:
        tar.extractall(path=outs_dir)
    print("cell_feature_matrix extracted")
else:
    print("Neither 'cell_feature_matrix.tar.gz' nor the extracted folder were found")

cell_feature_matrix extracted


#### 3. Clone the Hover-Net GitHub repository and follow installation steps

In this example data processing code we are using Hover-Net (Graham et al., Medical Image Analysis 2019) to segment nuclei in H&E images. Please clone their repository (https://github.com/vqdang/hover_net) and install their environment (separate to the environment for GHIST). Download their checkpoint `hovernet_original_consep_notype_tf2pytorch.tar` and place in the main `hover_net` folder.

## Setup - Parameters

Here, we define locations of data directories and files, and other parameters.

- ``code_dir``: path to the ``data_processing`` folder in this repo
- ``dir_output``: this is the directory that will be created where processed data will be saved
- ``dir_xenium_outs``: path to the extracted Xenium output bundle folder
- ``fp_he_img``: path to the H&E image file that is aligned to the Xenium data (``.ome.tif`` or ``.tif`` file)
- ``dir_hovernet``: path to the Hover-Net code (folder should be ``hover_net``)
- ``gpu_id``: which GPU to use
- ``n_processes``: how many CPUs to use

<span style="color: red; font-weight: bold">Please update the following parameters:</span>

The key parameter to update for this tutorial is `dir_hovernet`

In [5]:
config = SimpleNamespace(
    code_dir="../data_processing",
    dir_output="../tutorials/data_processing_demo_outputs",
    dir_xenium_outs="../tutorials/data_processing_demo",
    fp_he_img="../tutorials/data_processing_demo/he_image.tif",
    dir_hovernet="/dskh/nobackup/helenf/hover_net",
    gpu_id=0,
    n_processes=24
)

In [7]:
# Change working directory
os.chdir(config.code_dir)

## Step 1

Get the nuclei segmentation image for Xenium data. This step extracts from nuclei the boundaries file in the data bundle.

In [8]:
command = f"""
python 1_get_xenium_nuclei_seg_image.py \
    --fp_boundaries {config.dir_xenium_outs}/nucleus_boundaries.csv.gz \
    --dir_output {config.dir_output} \
    --fp_he_img {config.fp_he_img} \
    --n_processes {config.n_processes}
"""
os.system(command)


 50%|█████     | 3/6 [00:00<00:00, 23.05it/s]

Combining crops
Total nuclei 5410
Deleting intermediate files



100%|██████████| 6/6 [00:00<00:00, 28.28it/s]


0

## Step 2

Get the cell gene matrix for the Xenium data.

In [9]:
command = f"""
python 2_get_xenium_cell_gene_matrix.py \
    --dir_feature_matrix {config.dir_xenium_outs}/cell_feature_matrix \
    --dir_output {config.dir_output}
"""
os.system(command)


Loading ../tutorials/data_processing_demo/cell_feature_matrix/barcodes.tsv
Loading ../tutorials/data_processing_demo/cell_feature_matrix/features.tsv
Saved ../tutorials/data_processing_demo_outputs/cell_gene_matrix.csv
Deleting intermediate files


0

## Step 3.1

(H&E nuclei segmentation) - divide the H&E image in to patches for nuclei segmentation.

In [15]:
command = f"""
python 3_segment_nuclei_he_image.py \
    --dir_output {config.dir_output} \
    --fp_he_img {config.fp_he_img} \
    --dir_hovernet {config.dir_hovernet} \
    --gpu_id {config.gpu_id} \
    --step 1
"""
os.system(command)


Done cropping


0

## Step 3.2

(H&E nuclei segmentation) - segment the nuclei using Hover-Net.

During this step, we have encountered the error below, though the segmentation still ran fine:
``Error: mkl-service + Intel(R) MKL: MKL_THREADING_LAYER=INTEL is incompatible with libgomp.so.1 library. Try to import numpy first or set the threading layer accordingly. Set MKL_SERVICE_FORCE_INTEL to force it.``

In [16]:
conda_base = subprocess.check_output("conda info --base", shell=True).decode().strip()

cmd = f"""
source {conda_base}/etc/profile.d/conda.sh && \
conda activate hovernet && \
python 3_segment_nuclei_he_image.py \
  --dir_output {config.dir_output} \
  --fp_he_img {config.fp_he_img} \
  --dir_hovernet {config.dir_hovernet} \
  --gpu_id {config.gpu_id} \
  --step 2
"""

subprocess.run(cmd, shell=True, executable="/bin/bash")

Error: mkl-service + Intel(R) MKL: MKL_THREADING_LAYER=INTEL is incompatible with libgomp.so.1 library.
	Try to import numpy first or set the threading layer accordingly. Set MKL_SERVICE_FORCE_INTEL to force it.
|2025-05-09|08:41:30.276| [INFO] .... Detect #GPUS: 1
Process Patches:   0%|                                   | 0/10 [00:00<?, ?it/s]Error: mkl-service + Intel(R) MKL: MKL_THREADING_LAYER=INTEL is incompatible with libgomp.so.1 library.
	Try to import numpy first or set the threading layer accordingly. Set MKL_SERVICE_FORCE_INTEL to force it.
Process Patches: 100%|##########################| 10/10 [00:34<00:00,  3.50s/it]
Error: mkl-service + Intel(R) MKL: MKL_THREADING_LAYER=INTEL is incompatible with libgomp.so.1 library.
	Try to import numpy first or set the threading layer accordingly. Set MKL_SERVICE_FORCE_INTEL to force it.
|2025-05-09|08:42:22.354| [INFO] ........................ Done Assembling 0_0
Error: mkl-service + Intel(R) MKL: MKL_THREADING_LAYER=INTEL is incompa

Make sure you are using the correct env for hovernet
/dskh/nobackup/helenf/project_GHIST/_GITHUB_restructure_branch/tutorials/data_processing_demo_outputs/he_image_nuclei_seg_crops
/dskh/nobackup/helenf/project_GHIST/_GITHUB_restructure_branch/tutorials/data_processing_demo_outputs/he_image_nuclei_seg_crops_hovernet
Num crops found: 4


CompletedProcess(args='\nsource /dskh/nobackup/helenf/miniconda3/etc/profile.d/conda.sh && conda activate hovernet && python 3_segment_nuclei_he_image.py   --dir_output ../tutorials/data_processing_demo_outputs   --fp_he_img ../tutorials/data_processing_demo/he_image.tif   --dir_hovernet /dskh/nobackup/helenf/hover_net   --gpu_id 0   --step 2\n', returncode=0)

If the above doesn't work, please open up a terminal in the `GHIST/data_processing` folder and run:

```sh
conda activate hovernet

python 3_segment_nuclei_he_image.py --dir_output data_processing_demo_outputs --fp_he_img ../tutorials/data_processing_demo/he_image.tif --dir_hovernet {/path/to/hover_net} --gpu_id 0 --step 2
```

## Step 3.3

(H&E nuclei segmentation) - combine the nuclei from patches to a whole image.

In [17]:
conda_base = subprocess.check_output("conda info --base", shell=True).decode().strip()

cmd = f"""
source {conda_base}/etc/profile.d/conda.sh && \
conda activate ghist && \
python 3_segment_nuclei_he_image.py \
  --dir_output {config.dir_output} \
  --fp_he_img {config.fp_he_img} \
  --dir_hovernet {config.dir_hovernet} \
  --gpu_id {config.gpu_id} \
  --step 3
"""

subprocess.run(cmd, shell=True, executable=f"/bin/bash")

Num IDs before combining: 5891
Combining crops...
Num nuclei intermediate step 5870
Num nuclei final: 5844
Saved ../tutorials/data_processing_demo_outputs/he_image_nuclei_seg.tif
Num nuclei final: 5840
Deleting intermediate files



100%|██████████| 1/1 [00:00<00:00, 135.52it/s]

100%|██████████| 1/1 [00:00<00:00, 68.25it/s]


CompletedProcess(args='\nsource /dskh/nobackup/helenf/miniconda3/etc/profile.d/conda.sh && conda activate ghist && python 3_segment_nuclei_he_image.py   --dir_output ../tutorials/data_processing_demo_outputs   --fp_he_img ../tutorials/data_processing_demo/he_image.tif   --dir_hovernet /dskh/nobackup/helenf/hover_net   --gpu_id 0   --step 3\n', returncode=0)

If the above doesn't work, please open up a terminal in the `GHIST/data_processing` folder and run:

```sh
conda activate ghist 

python 3_segment_nuclei_he_image.py --dir_output data_processing_demo_outputs --fp_he_img ../tutorials/data_processing_demo/he_image.tif --dir_hovernet {/path/to/hover_net} --gpu_id 0 --step 3
```

## Step 4

Determine corresponding cells between H&E images and Xenium data.

In [18]:
command = f"""
python 4_get_corresponding_cells.py \
    --dir_output {config.dir_output} \
    --n_processes {config.n_processes}
"""
os.system(command)


Processing using CPUs: 24


100%|██████████| 5841/5841 [00:00<00:00, 9120.32it/s]


Saved ../tutorials/data_processing_demo_outputs/cell_gene_matrix_filtered.csv


0

📌 At this point the following files should in the directory ``dir_output``:

- ``cell_gene_matrix.csv`` - cell gene matrix for all cells in the Xenium data
- *``cell_gene_matrix_filtered.csv`` - cell gene matrix only for filtered cells
- ``genes.txt`` - list of genes in the panel
- *``he_image_nuclei_seg.tif`` - nuclei segmented from the H&E image
- ``he_image_nuclei_seg_microns.tif`` - nuclei segmented from the H&E image resized to 1 pixel = 1 micron
- ``matched_nuclei.csv`` - corresponding cell IDs between H&E and Xenium
- *``matched_nuclei_filtered.csv`` - corresponding cell IDs of filtered cells
- ``xenium_cell_ids_dict.csv`` - created only if Xenium cell IDs are strings
- ``xenium_nuclei.tif`` - nuclei segmented from Xenium data

(*) marks files needed to run GHIST (see ``data_demo`` as an example)

## Optional - Cell type

Cell type annotation - if you want to use cell type or neighbourhood information, run your preferred cell annotation method on ``cell_gene_matrix_filtered.csv`` to get cell type labels. Then create a csv file with columns like the following (e.g., see ``data_demo/celltype_filtered.csv``):

```
c_id,ct
1,T
2,Malignant
3,Macrophage
...etc
```

Number of cell types: based on our experiments we recommend using <10 cell types

📌 If you do not wish to use cell type information, in your config file:

- ``"celltype": false,``
- delete parameter `fp_cell_type` 

## Optional - AvgExp

Averaged single cell gene expression profiles of cell types - this is an optional input. These profiles do not need to be matched for the same sample of interest, but should ideally be for the same tissue type (e.g. breast cancer). AvgExp can have a flexible number and categories of cell types from a single or multiple reference datasets. The same AvgExp data should be used during training and inference.

📌 If you do not wish to use AvgExp, in your config file:

- ``"avgexp": false,``
- delete parameter `fp_avgexp` 

# New territory

## Step 5

Create variant data with varseek

In [ ]:
dry_run = True

if not os.path.exists(config.dir_output):
    config.dir_output = "../data_demo"  # only if I am jumping straight here

variant_data_dir = os.path.join(config.dir_output, "variant_data")
if not os.path.exists(variant_data_dir):
    os.makedirs(variant_data_dir)

GEO_url = "https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE243280"
SRA_url = "https://www.ncbi.nlm.nih.gov/Traces/study/?acc=PRJNA1017511&f=librarysource_s%3An%3Atranscriptomic%2520single%2520cell&o=acc_s%3Aa&s=SRR26065624,SRR26065625,SRR26065626,SRR26065627,SRR26065628,SRR26065629,SRR26065630,SRR26065631,SRR26065632,SRR26065633,SRR26065634,SRR26065635,SRR26065636,SRR26065637,SRR26065638,SRR26065639"
SRA_accession_file = os.path.join(variant_data_dir, "SRR_Acc_List_scrnaseq_tutorial.txt")  #!!! this is wrong - replace with new

sequencing_data_raw_dir = os.path.join(variant_data_dir, "sequencing_data_raw")
if not os.path.exists(sequencing_data_raw_dir) or len(os.listdir(sequencing_data_raw_dir)) == 0:
    if not os.path.exists(SRA_accession_file):
        raise ValueError(f"Please download the SRA accession file and save it as {SRA_accession_file}\nSRA url: {SRA_url}\nGEO url: {GEO_url}")
    os.makedirs(sequencing_data_raw_dir, exist_ok=True)

    # check for fastqs
    with open(SRA_accession_file) as f:
        srr_list = f.read().split()
    try:
        print("Downloading fastq files (152GB for .fastq files)")
        
        print("Running prefetch")
        prefetch_command = ["prefetch", "--progress", "--verbose"] + srr_list
        print(" ".join(prefetch_command))
        if not dry_run:
            subprocess.run(prefetch_command, check=True)
        
        print("Downloading files")
        data_download_command = ["fasterq-dump", "--outdir", os.path.abspath(sequencing_data_raw_dir), "--threads", str(config.n_processes), "--progress", "--verbose", "--split-files"] + srr_list
        print(" ".join(data_download_command))
        if not dry_run:
            subprocess.run(data_download_command, check=True)
        
        print(f"Fastq data downloaded successfully to {sequencing_data_raw_dir}")
    except Exception as e:
        print(f"Error running fasterq-dump: {e}")
        raise  # re-raises the original exception
else:
    print(f"Fastq data already exists in {sequencing_data_raw_dir}.")

varseek_cosmic_cmc_index_url = "https://caltech.box.com/shared/static/f58qvvmp6en4dha28d7pakafg8j9jzqi.idx"
varseek_cosmic_cmc_t2g_url = "https://caltech.box.com/shared/static/taqo1cyswui69rdiy3jqpxdtsgbnin10.txt"
vk_ref_dir = os.path.join(variant_data_dir, "vk_ref_out")
idx_path = os.path.join(vk_ref_dir, "cosmic_cmc_index.idx")
t2g_path = os.path.join(vk_ref_dir, "cosmic_cmc_t2g.txt")

os.makedirs(vk_ref_dir, exist_ok=True)    
if not os.path.exists(idx_path):
    subprocess.run(f"wget {varseek_cosmic_cmc_index_url} -O {idx_path}", shell=True, check=True)
if not os.path.exists(t2g_path):
    subprocess.run(f"wget {varseek_cosmic_cmc_t2g_url} -O {t2g_path}", shell=True, check=True)

Running prefetch
prefetch --progress --verbose SRR26065624 SRR26065625 SRR26065626 SRR26065627 SRR26065628 SRR26065629 SRR26065630 SRR26065631 SRR26065632 SRR26065633 SRR26065634 SRR26065635 SRR26065636 SRR26065637 SRR26065638 SRR26065639
fasterq-dump --outdir ../data_demo/variant_data/sequencing_data_raw --threads 24 --progress --verbose --split-files SRR26065624 SRR26065625 SRR26065626 SRR26065627 SRR26065628 SRR26065629 SRR26065630 SRR26065631 SRR26065632 SRR26065633 SRR26065634 SRR26065635 SRR26065636 SRR26065637 SRR26065638 SRR26065639
Fastq data downloaded successfully to ../data_demo/variant_data/sequencing_data_raw


In [ ]:
technology = "10XV3"
k = 51  #* don't change unless I change my vk ref index settings
min_counts = 3
use_binary_matrix = True
drop_empty_columns = True

command = f"""
python 5_get_variant_data.py \
    --dir_output {config.dir_output} \
    --technology {technology} \
    --k {k} \
    --min_counts {min_counts} \
    --disable_use_binary_matrix {not use_binary_matrix} \
    --disable_drop_empty_columns {not drop_empty_columns} \
    --n_processes {config.n_processes} \
    --variant_data_dir {variant_data_dir} \
    --fastqs_dir {sequencing_data_raw_dir} \
    --vk_ref_dir {vk_ref_dir} \
    --index {idx_path} \
    --t2g {t2g_path}
"""
os.system(command)